In [1]:
!pip install transformers langchain chromadb sentence-transformers pypdf2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 9.1 MB/s eta 

In [4]:
from google.colab import files
uploaded = files.upload()


Saving companyprofile.pdf to companyprofile.pdf


In [5]:
from PyPDF2 import PdfReader

reader = PdfReader("companyprofile.pdf")  # make sure this matches exactly
text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview first 1000 characters


 
For all your Insurance Needs  
 
 
Chai House,5th Floor,  
Koinange Street,  
P.O BOX 10150 GPO 00100,  
Nairobi,  Kenya  Majani Plaza  
Company HISTORY  
Majani Insurance Brokers Ltd is a wholly owned subsidi-
ary of the Kenya Tea Development Agency LTD having 
started as a department in 1976 to serve KTDA insurance 
needs.  
 
In 1998, Majani Insurance Brokers transformed into a 
composite Insurance Broker serving both KTDA and non -
KTDA markets.  
 
Majani is a top broker in the industry based on its turno-
ver, profitability and strong financial base.  
 
Majani Insurance Brokers Attained ISO certification in 
December 2007, making it a pioneer and the only ISO 
certified insurance broker at that time.  
 
Majani Insurance Brokers is a market leader in micro in-
surance having come up with a low cost medical product 
targeting the low income earners and unemployed persons 
within the republic a first in the insurance sector.  
 Vision  
To be the leading provider of Insurance Br

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # each chunk will have about 500 characters
    chunk_overlap=100    # small overlap so no information is lost
)

# Split the text
chunks = splitter.split_text(text)

print(f"Total chunks created: {len(chunks)}")
print("\nFirst chunk preview:\n", chunks[0])


Total chunks created: 10

First chunk preview:
 For all your Insurance Needs  
 
 
Chai House,5th Floor,  
Koinange Street,  
P.O BOX 10150 GPO 00100,  
Nairobi,  Kenya  Majani Plaza  
Company HISTORY  
Majani Insurance Brokers Ltd is a wholly owned subsidi-
ary of the Kenya Tea Development Agency LTD having 
started as a department in 1976 to serve KTDA insurance 
needs.  
 
In 1998, Majani Insurance Brokers transformed into a 
composite Insurance Broker serving both KTDA and non -
KTDA markets.


In [8]:
!pip install -q sentence-transformers chromadb pypdf2


In [9]:
# Directly create embeddings using sentence-transformers and store in Chroma
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

# load the SBERT model
sbert = SentenceTransformer("all-MiniLM-L6-v2")

# convert chunks (list of strings) into embeddings
emb_list = sbert.encode(chunks, show_progress_bar=True, convert_to_numpy=True)

# create a Chroma client and collection
client = chromadb.Client()
collection = client.create_collection(name="company_docs_embeddings")

# add documents and embeddings (use simple ids)
ids = [f"chunk_{i}" for i in range(len(chunks))]
collection.add(documents=chunks, embeddings=emb_list.tolist(), ids=ids)

print("✅ Embeddings added to Chroma (direct method).")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Embeddings added to Chroma (direct method).


In [10]:
from transformers import pipeline

# Load a free question-answering model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def ask_question(query):
    # Step 1: find relevant chunks
    query_emb = sbert.encode([query])[0].tolist()
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3,
        include=["documents"]
    )

    # Step 2: combine top chunks into context
    context = " ".join(results["documents"][0])

    # Step 3: ask the QA model
    answer = qa_pipeline(question=query, context=context)

    print(f"Question: {query}")
    print(f"Answer: {answer['answer']}")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [11]:
ask_question("Where is the head office located?")
ask_question("What services does the company offer?")


Question: Where is the head office located?
Answer: Chai House –  Koinange Street
Question: What services does the company offer?
Answer: exceptional Insurance Brokerage services
